# ML Modeling

## Libraries

In [139]:
# main libraries
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# metrics
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score

# ML classifier models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

# resampling
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from collections import Counter

# model selection (CV)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from scipy.stats import randint

## Machine Learning

### Separating the DF into X and Y

In [2]:
bank = pd.read_csv("../data/bank_processed_data.csv", index_col=0)
bank.head()

,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,...,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Education_Level_encoded,Income_Category_encoded,Card_Category_encoded,x0_Married,x0_Single,x0_Unknown,x1_Existing Customer,x2_M
0,45,3,39,5,1,3,12691.0,777,1.335,1144,...,1.625,0.061,2.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0
1,49,5,44,6,1,2,8256.0,864,1.541,1291,...,3.714,0.105,4.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
2,51,3,36,4,1,0,3418.0,0,2.594,1887,...,2.333,0.000,4.0,4.0,0.0,1.0,0.0,0.0,1.0,1.0
3,40,4,34,3,4,1,3313.0,2517,1.405,1171,...,2.333,0.760,2.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
4,40,3,21,5,1,0,4716.0,0,2.175,816,...,2.500,0.000,1.0,3.0,0.0,1.0,0.0,0.0,1.0,1.0


In [3]:
# separating X and y
X = bank.drop(columns="x1_Existing Customer")
y = bank["x1_Existing Customer"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state = 42)

In [4]:
# Scaling the data

scaler = StandardScaler() # initialize the scaler

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Choosing the Models

The column that I want to identify is **x1_Existing Customer**, being **1** if the customer is still a customer, and **0** otherwise. In this case, as it's a True/False decission, the models that fit better for this type of Supervised ML are the Classifiers.

I will start checking the different models, without parameter tuning, for identify which are the models that perform better.

In [5]:
# Initializing the models

neigh = KNeighborsClassifier()
tree = DecisionTreeClassifier()
gradient = GradientBoostingClassifier()
RF = RandomForestClassifier()
adaboost = AdaBoostClassifier()
extra_tree = ExtraTreesClassifier()
support_vector = SVC()


models = [neigh, tree, RF, adaboost, gradient, extra_tree, support_vector]
model_names = ["KNeighbors", "DecisionTree", "RandomForest", "AdaBoost", 
               "GradientBoost", "ExtraTress", "SVC"]

The data per each category is not balanced, as customers represent 83.8% of the sample, the accuracy here is not relevant. 

In this scenario, I will focus more on `recall`, to ensure that the model classifies correctly the labels, and the precision. Mention that, the main focus will be on the label **0.0, as it is the customers that already churned the bank, and we want to focus on that part** to ensure that our model is able to predict possible future cases and act before churn happens.

Last, but not least, `macro avg` will also be taken into consideration, as we want to ensure that **0.0** are classified correctly, but we want that the amount of **1.0** are good too. I would have to find the perfect balance between those metrics.

*NOTES*

The `precision` is the ratio TP / (TP + FP) where TP is the number of true positives and FP the number of false positives. The precision is intuitively **the ability of the classifier not to label as positive a sample that is negative**.

The `recall` is the ratio TP / (TP + FN) where TP is the number of true positives and FN the number of false negatives. The recall is intuitively **the ability of the classifier to find all the positive samples**. Note that in binary classification, recall of the positive class is also known as `sensitivity`; recall of the negative class is `specificity`.

The most important for this work is to increase the **sensitivity** (to detect all churn cases). Even though the other parameters are very important also.

#### Finding the best classification model

In [6]:
def top3_classifier_model(models):
    """
    Input: Models to test
    Output: DF top 3 models
    """
    # first batch of empty lists    
    time_to_train = []
    accuracy = []
    macro_precision = []
    macro_recall = []
    macro_F1 = []
    report_dict = []

    for model in models:    
        start = time.time()
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)

        # metrics
        accuracy_ = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average="macro")
        precision = precision_score(y_test, y_pred, average="macro")
        f1 = f1_score(y_test, y_pred, average="macro")
        clasf_report_dict = classification_report(y_test, y_pred, output_dict=True)

        # appending to empty lists
        time_to_train.append((time.time() - start))
        accuracy.append(round(accuracy_,4))
        macro_precision.append(round(precision,4))
        macro_recall.append(round(recall,4))
        macro_F1.append(round(f1,4))
        report_dict.append(clasf_report_dict)
    
    # second batch
    precision_0 = []
    recall_0 = []
    f1_0 = []
    precision_1 = []
    recall_1 = []
    f1_1 = []

    for report in report_dict:
        # Info of churn label
        precision_0.append(round(report["0.0"]["precision"],4))
        recall_0.append(round(report["0.0"]["recall"],4))
        f1_0.append(round(report["0.0"]["f1-score"],4))

        # Info of current customers
        precision_1.append(round(report["1.0"]["precision"],4))
        recall_1.append(round(report["1.0"]["recall"],4))
        f1_1.append(round(report["1.0"]["f1-score"],4))
        
    # creating DF
    best_models_DF = pd.DataFrame({"model":model_names,
                                   "training_time":time_to_train,
                                   "accuracy":accuracy,
                                   "precision_macro":macro_precision,
                                   "recall_macro":macro_recall,
                                   "f1_macro":macro_F1,
                                   "precision_0":precision_0,
                                   "recall_0":recall_0,
                                   "f1_0":f1_0,
                                   "precision_1":precision_1,
                                   "recall_1":recall_1,
                                   "f1_1":f1_1
                                  })
    
    # getting top 3 models
    top3 = best_models_DF.sort_values(by=["f1_macro"], ascending=False).reset_index(drop=True).iloc[:3]
    
    return top3

In [8]:
top3_classifier_model(models)

,model,training_time,accuracy,precision_macro,recall_macro,f1_macro,precision_0,recall_0,f1_0,precision_1,recall_1,f1_1
0,GradientBoost,1.475824,0.9650,0.9478,0.9194,0.9328,0.9233,0.8523,0.8864,0.9722,0.9865,0.9793
1,RandomForest,0.866706,0.9630,0.9461,0.9132,0.9287,0.9223,0.8400,0.8792,0.9699,0.9865,0.9781
2,AdaBoost,0.386940,0.9605,0.9332,0.9180,0.9254,0.8939,0.8554,0.8742,0.9726,0.9806,0.9766


The three models that does the best selection for the churned customers and also for the actual customers are **GradientBoost**, **AdaBoost** and **RandomForest**.

Now that we have in mind which models work best, let's start tuning them for improve their results

### Tuning Models

#### GradientBoost

In [63]:
# start_time = time.time()

# gradient_params = {"loss":["deviance", "exponential"],
#                   "criterion":["friedman_mse", "mse", "mae"],
#                   "max_features":["auto", "sqrt", "log2"],
#                   "n_estimators":randint(low=50, high=300),
#                   "max_depth":randint(low=2, high=8),
#                   "max_leaf_nodes":randint(low=5, high=15)
#                   }

# scorers = {"precision_score": make_scorer(precision_score, average="macro"),
#            "recall_score": make_scorer(recall_score, average="macro"),
#            "f1_score": make_scorer(f1_score, average="macro")
#           }

# gradient_search = RandomizedSearchCV(gradient,
#                                      gradient_params,
#                                      n_iter=10,
#                                      n_jobs=-1,
#                                      cv=StratifiedKFold(),
#                                      scoring=scorers,
#                                      refit=False,
#                                      random_state=42
#                                     )

# gradient_search.fit(X_train_scaled, y_train)

# print("--- %s seconds ---" % (time.time() - start_time))

After the results obtained, I will create a DF for visualize which are the scores for each scoring. After that, I will pick the parameters that performed better for passing it to GridSearchCV.

In [64]:
# gradient_results = pd.DataFrame(gradient_search.cv_results_)

# gradient_results = gradient_results[["params", "mean_test_precision_score", "rank_test_precision_score",
#                                      "mean_test_recall_score", "rank_test_recall_score",
#                                      "mean_test_f1_score", "rank_test_f1_score"]]

# gradient_results

In [65]:
# gradient_results["params"][4]

When we know which are the best parameters to pass to the model, we will create a function for each one of them and then obtain their respective **y_pred**. The **y_value** obtained will be stored in a variable for, later on, build the `classification_report` and `confusion_matrix`.

The structure will be the same for future models and variations.

In [62]:
def gradient_gridsearch(X_train, y_train):
    """
    Input: X_train and y_train for model training
    Output: Best params
    
    """
    # Inputting the different parameters obtained from the RandomizedSearchCV
    gradient_params = {"loss":["deviance", "exponential"],
                       "criterion":["friedman_mse", "mse"],
                       "max_features":["log2","sqrt"],
                       "n_estimators":[260, 265, 270, 275, 280],
                       "max_depth":[3, 4, 5],
                       "max_leaf_nodes":[10, 12, 14, 16]
                      }

    # Running the best_gradient with GridSearchCV
    best_gradient = GridSearchCV(gradient,
                                 gradient_params,
                                 n_jobs=-1,
                                 cv=StratifiedKFold(),
                                 scoring="f1_macro"
                                )
    
    best_gradient.fit(X_train, y_train)
    
    best = best_gradient.best_estimator_
    
    return best

In [63]:
gradient_gridsearch(X_train_scaled, y_train)

GradientBoostingClassifier(max_depth=5, max_features='log2', max_leaf_nodes=14,
                           n_estimators=260)

In [145]:
def gradient_trainer(X_train, X_test, y_train, y_test, loss, criterion, max_features, n_estimators, max_depth, max_leaf_nodes):
    """
    Input: All the inputs for training the GradientBoostingClassifier with GridSearchCV
    Output: y_pred
    """
    # Creating the best_gradient 
    best_gradient = GradientBoostingClassifier(loss=loss, criterion=criterion,
                                               max_features=max_features, n_estimators=n_estimators,
                                               max_depth=max_depth, max_leaf_nodes=max_leaf_nodes
                                              )
    
    # Model fit
    best_gradient.fit(X_train, y_train)
    
    # Model train
    y_pred = best_gradient.predict(X_test)
    
    # Getting classification report
    c_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=['Churn', 'Customer']))
    
    # Getting Kappa score
    kappa = cohen_kappa_score(y_test, y_pred)
    
    return y_pred, c_report, kappa

In [146]:
y_pred_gradient, classification_report_gradient, kappa_gradient = gradient_trainer(X_train_scaled,
                                                                                   X_test_scaled,
                                                                                   y_train,
                                                                                   y_test,
                                                                                   "deviance",
                                                                                   "friedman_mse",
                                                                                   "log2",
                                                                                   265,
                                                                                   5,
                                                                                   14
                                                                                  )

#### AdaBoost

In [13]:
# start_time = time.time()

# adaboost_params = {"algorithm":["SAMME", "SAMME.R"],
#                   "n_estimators":randint(low=10, high=200)
#                   }

# scorers = {"precision_score": make_scorer(precision_score, average="macro"),
#            "recall_score": make_scorer(recall_score, average="macro"),
#            "f1_score": make_scorer(f1_score, average="macro")
#           }

# adaboost_search = RandomizedSearchCV(adaboost,
#                                      adaboost_params,
#                                      n_iter=10,
#                                      n_jobs=-1,
#                                      cv=10,
#                                      scoring=scorers,
#                                      refit=False,
#                                      random_state=42
#                                     )

# adaboost_search.fit(X_train_scaled, y_train)

# print("--- %s seconds ---" % (time.time() - start_time))

In [14]:
# adaboost_results = pd.DataFrame(adaboost_search.cv_results_)

# adaboost_results = adaboost_results[["params", "mean_test_precision_score", "rank_test_precision_score",
#                                      "mean_test_recall_score", "rank_test_recall_score",
#                                      "mean_test_f1_score", "rank_test_f1_score"]]

# adaboost_results

In [15]:
# adaboost_results["params"][8]

In [42]:
def adaboost_gridsearch(X_train, y_train):
    """
    Input: X_train and y_train for model training
    Output: Best params
    
    """
    # Inputting the different parameters obtained from the RandomizedSearchCV
    adaboost_params = {"algorithm":["SAMME", "SAMME.R"],
                       "n_estimators":[155, 158, 160, 161, 164]
                      }

    # Running the best_gradient with GridSearchCV
    best_adaboost = GridSearchCV(adaboost,
                                 adaboost_params,
                                 n_jobs=-1,
                                 cv=StratifiedKFold(),
                                 scoring="f1_macro"
                                )
    
    best_adaboost.fit(X_train, y_train)
    
    best = best_adaboost.best_estimator_
    
    return best

In [43]:
adaboost_gridsearch(X_train_scaled, y_train)

AdaBoostClassifier(n_estimators=155)

In [150]:
def adaboost_trainer(X_train, X_test, y_train, y_test, algorithm, n_estimators):
    """
    Input: All the inputs for training the AdaBoostClassifier with GridSearchCV
    Output: y_pred
    """
    # Creating the best_adaboost 
    best_adaboost = AdaBoostClassifier(algorithm=algorithm, n_estimators=n_estimators)
    
    # Model fit
    best_adaboost.fit(X_train, y_train)
    
    # Model train
    y_pred = best_adaboost.predict(X_test)
    
    # Getting classification report
    c_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=['Churn', 'Customer']))
    
    # Getting Kappa score
    kappa = cohen_kappa_score(y_test, y_pred)
    
    return y_pred, c_report, kappa

In [151]:
y_pred_adaboost, classification_report_adaboost, kappa_adaboost = adaboost_trainer(X_train_scaled,
                                                                                   X_test_scaled,
                                                                                   y_train,
                                                                                   y_test,
                                                                                   "SAMME.R",
                                                                                   155
                                                                                  )

#### RandomForest

In [18]:
# start_time = time.time()

# RF_params = {"criterion":["gini", "entropy"],
#              "max_features":["auto", "sqrt", "log2"],
#              "class_weight":["balanced", "balanced_subsample"],          
#              "n_estimators":randint(low=10, high=400),
#              "max_depth":randint(low=2, high=20),
#              "min_samples_split":randint(low=2, high=40)
#             }

# scorers = {"precision_score": make_scorer(precision_score, average="macro"),
#            "recall_score": make_scorer(recall_score, average="macro"),
#            "f1_score": make_scorer(f1_score, average="macro")
#           }

# RF_search = RandomizedSearchCV(RF,
#                                RF_params,
#                                n_iter=10,
#                                n_jobs=-1,
#                                cv=50,
#                                scoring=scorers,
#                                refit=False,
#                                random_state=42)

# RF_search.fit(X_train_scaled, y_train)

# print("--- %s seconds ---" % (time.time() - start_time))

In [19]:
# RF_results = pd.DataFrame(RF_search.cv_results_)

# RF_results = RF_results[["params", "mean_test_precision_score", "rank_test_precision_score",
#                          "mean_test_recall_score", "rank_test_recall_score",
#                          "mean_test_f1_score", "rank_test_f1_score"]]

# RF_results

In [20]:
# RF_results["params"][6]

In [50]:
def RF_gridsearch(X_train, y_train):
    """
    Input: X_train and y_train for model training
    Output: Best params
    
    """
    # Inputting the different parameters obtained from the RandomizedSearchCV    
    RF_params = {"criterion":["gini", "entropy"],
                 "max_features":["auto", "sqrt", "log2"],
                 "class_weight":["balanced", "balanced_subsample"],               
                 "n_estimators":[390, 395, 400, 405],
                 "max_depth":[8, 10, 12],
                 "min_samples_split":[16, 18, 20, 22]
                }

    # Running the best_gradient with GridSearchCV
    best_RF = GridSearchCV(RF,
                                 RF_params,
                                 n_jobs=-1,
                                 cv=StratifiedKFold(),
                                 scoring="f1_macro"
                                )
    
    best_RF.fit(X_train, y_train)
    
    best = best_RF.best_estimator_
    
    return best

In [51]:
RF_gridsearch(X_train_scaled, y_train)

RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=12, min_samples_split=18, n_estimators=390)

In [153]:
def RF_trainer(X_train, X_test, y_train, y_test, class_weight, criterion, max_features, max_depth, min_samples_split, n_estimators):
    """
    Input: All the inputs for training the RandomForestClassifier with GridSearchCV
    Output: y_pred
    """
    # Creating the best_RF
    best_RF = RandomForestClassifier(class_weight=class_weight, criterion=criterion, max_features=max_features,
                                     max_depth=max_depth, min_samples_split=min_samples_split, n_estimators=n_estimators)
    
    # Model fit
    best_RF.fit(X_train, y_train)
    
    # Model train
    y_pred = best_RF.predict(X_test)
    
    # Getting classification report
    c_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True, target_names=['Churn', 'Customer']))
    
    # Getting Kappa score
    kappa = cohen_kappa_score(y_test, y_pred)
    
    return y_pred, c_report, kappa

In [154]:
y_pred_RF, classification_report_RF, kappa_RF = RF_trainer(X_train_scaled,
                                                           X_test_scaled,
                                                           y_train,
                                                           y_test,
                                                           "balanced",
                                                           "entropy",
                                                           "auto",
                                                           12,
                                                           18,
                                                           390
                                                          )

### Analyzing the results

In [66]:
def initial_predictions(X_train, y_train, X_test):
    """
    Input: Variables for training a model
    Output: y_pred for three different models    
    """
    # GradientBoost
    gradient.fit(X_train, y_train)
    y_pred_initial_gradient = gradient.predict(X_test)
    
    # AdaBoost
    adaboost.fit(X_train, y_train)
    y_pred_initial_adaboost = adaboost.predict(X_test)
    
    # RF
    RF.fit(X_train, y_train)
    y_pred_initial_RF = RF.predict(X_test)
    
    return y_pred_initial_gradient, y_pred_initial_adaboost, y_pred_initial_RF

In [67]:
y_pred_initial_gradient, y_pred_initial_adaboost, y_pred_initial_RF = initial_predictions(X_train_scaled, 
                                                                                          y_train, 
                                                                                          X_test_scaled
                                                                                         )

In [127]:
def initial_class_reports(y_test, y_pred1, y_pred2, y_pred3):
    """
    Input: Three y_pred variables of the different models
    Output: Classification reports of the models    
    """
    # GradientBoost
    initial_classification_gradient = pd.DataFrame(classification_report(y_test, y_pred1, output_dict=True, target_names=['Churn', 'Customer']))
    
    # AdaBoost
    initial_classification_adaboost = pd.DataFrame(classification_report(y_test, y_pred2, output_dict=True, target_names=['Churn', 'Customer']))
    
    # RF
    initial_classification_RF = pd.DataFrame(classification_report(y_test, y_pred3, output_dict=True, target_names=['Churn', 'Customer']))
    
    return initial_classification_gradient, initial_classification_adaboost, initial_classification_RF

In [128]:
first_class_gradient, first_class_adaboost, first_class_RF = initial_class_reports(y_test,
                                                                                  y_pred_initial_gradient,
                                                                                  y_pred_initial_adaboost,
                                                                                  y_pred_initial_RF
                                                                                  )

In [164]:
# Comparing GradientBoost
print(f"GradientBoost\nInitial:\n{first_class_gradient}\n")
print(f"Modified:\n{classification_report_gradient}\n\n")

# Comparing AdaBoost
print(f"AdaBoost\nInitial:\n{first_class_adaboost}\n")
print(f"Modified:\n{classification_report_adaboost}\n\n")

# Comparing RandomForest
print(f"RandomForest\nInitial:\n{first_class_RF}\n")
print(f"Modified:\n{classification_report_RF}\n\n")

# Looking Kappa
print(f"GradientBoost Kappa: {round(kappa_gradient,4)}")
print(f"AdaBoost Kappa: {round(kappa_adaboost,4)}")
print(f"RF Kappa: {round(kappa_RF,4)}")

GradientBoost
Initial:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.923333     0.972190  0.964956     0.947762      0.964353
recall       0.852308     0.986479  0.964956     0.919393      0.964956
f1-score     0.886400     0.979282  0.964956     0.932841      0.964383
support    325.000000  1701.000000  0.964956  2026.000000   2026.000000

Modified:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.948052     0.980792  0.975814     0.964422      0.975540
recall       0.898462     0.990594  0.975814     0.944528      0.975814
f1-score     0.922591     0.985668  0.975814     0.954130      0.975550
support    325.000000  1701.000000  0.975814  2026.000000   2026.000000


AdaBoost
Initial:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.893891     0.972595  0.960513     0.933243      0.959969
recall       0.855385     0.980600  0.960513     0.917992      0.960513
f1-score  

Looking into the `classification_report`, we will focus on the **Churn** column. On a first view, we can identify that the best models are the `GradientBoost` and `RandomForest`. Let's dive deep and compare each other:
1. **`GradientBoost`**: The `precision` is 0.9480, the `recall` is 0.8984, the `f1-score` is 0.9225 and the `accuracy` is 0.9758.
2. **`RandomForest`**: The `precision` is 0.8468, the `recall` is 0.9015, the `f1-score` is 0.8733 and the `accuracy` is 0.9580.

Although the `recall` is slightly higher on the `RandomForest`, the precision has a big difference compared to the `GradientBoost`. Also, taking into consideration that the `f1-score` will give us a more accurate reference of which model performs better with a more conservative approach, we can ensure that the **`GradientBoost`** is the best model overall.

We can also notice that the **Customer** column is much better on the `GradientBoost` rather than on the `RandomForest`, and, lastly, there is a considerable difference of `cohen_kappa_score` between both models.

For further testing, we will only focus with **`GradientBoost`**.

### Changing train_test_split

As we already did our initial `RandomizedSearchCV` for obtaining the values were we should focus with the `GridSearchCV`, in this case is not neccesary to do that step again.

What we will do, is the following:
* Split the test size in two different values, **0.1** and **0.3**.
* Obtain the best parameters for the gridsearch and see if they are slightly different from the previous model.
* Train the models with the new samples.
* Compare the new `classification_report` with the previous one and see if there is improvement. **In case that the model improves, for further analysis we will use the best one**.

#### 10% Test Size

In [79]:
X_train_10, X_test_10, y_train_10, y_test_10 = train_test_split(X, y, test_size = 0.1, stratify=y ,random_state = 42)

In [80]:
# Scaling the data
X_train_scaled_10 = scaler.fit_transform(X_train_10)
X_test_scaled_10 = scaler.transform(X_test_10)

In [81]:
# Obtaining best params
gradient_gridsearch(X_train_scaled_10, y_train_10)

GradientBoostingClassifier(criterion='mse', max_depth=5, max_features='sqrt',
                           max_leaf_nodes=16, n_estimators=265)

In [166]:
# Storing y_pred
y_pred_gradient_10, class_report_gradient_10, kappa_gradient_10 = gradient_trainer(X_train_scaled_10,
                                                                                   X_test_scaled_10,
                                                                                   y_train_10,
                                                                                   y_test_10,
                                                                                   "deviance",
                                                                                   "mse",
                                                                                   "sqrt",
                                                                                   265,
                                                                                   5,
                                                                                   16
                                                                                  )

#### 30% Test Size

In [83]:
X_train_30, X_test_30, y_train_30, y_test_30 = train_test_split(X, y, test_size = 0.3, stratify=y ,random_state = 42)

In [84]:
# Scaling the data
X_train_scaled_30 = scaler.fit_transform(X_train_30)
X_test_scaled_30 = scaler.transform(X_test_30)

In [85]:
# Obtaining best params
gradient_gridsearch(X_train_scaled_30, y_train_30)

GradientBoostingClassifier(max_depth=5, max_features='log2', max_leaf_nodes=14,
                           n_estimators=270)

In [163]:
# Storing y_pred
y_pred_gradient_30, class_report_gradient_30, kappa_gradient_30 = gradient_trainer(X_train_scaled_30,
                                                                                   X_test_scaled_30,
                                                                                   y_train_30,
                                                                                   y_test_30,
                                                                                   "deviance",
                                                                                   "friedman_mse",
                                                                                   "log2",
                                                                                   270,
                                                                                   5,
                                                                                   14
                                                                                  )

#### New sample results VS Previous sample

In [167]:
# Comparing GradientBoost
print(f"GradientBoost\ntest_size=0.2:\n{classification_report_gradient}\n")
print(f"test_size=0.1:\n{class_report_gradient_10}\n")
print(f"test_size=0.3:\n{class_report_gradient_30}\n\n")

print(f"test_size=0.2 Kappa: {round(kappa_gradient,4)}")
print(f"test_size=0.1 Kappa: {round(kappa_gradient_10,4)}")
print(f"test_size=0.3 Kappa: {round(kappa_gradient_30,4)}")

GradientBoost
test_size=0.2:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.948052     0.980792  0.975814     0.964422      0.975540
recall       0.898462     0.990594  0.975814     0.944528      0.975814
f1-score     0.922591     0.985668  0.975814     0.954130      0.975550
support    325.000000  1701.000000  0.975814  2026.000000   2026.000000

test_size=0.1:
                Churn    Customer  accuracy    macro avg  weighted avg
precision    0.920732    0.985866  0.975321     0.953299      0.975385
recall       0.926380    0.984706  0.975321     0.955543      0.975321
f1-score     0.923547    0.985285  0.975321     0.954416      0.975351
support    163.000000  850.000000  0.975321  1013.000000   1013.000000

test_size=0.3:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.939732     0.974141  0.969069     0.956937      0.968616
recall       0.862705     0.989416  0.969069     0.926060      0.969069
f1-score

* **`test_size` reduced**

Looking on the *Churn* customers, when the `test_size` is reduced, the `recall` improves and the `precision` goes worse.

The `recall` for the churned customers improved considerable, above the 0.9, and although that the overall accuracy dropped a little bit, the `f1-score` increased, which offers a more stable model.


* **`test_size` amplified**

When we increase the `test_size`, the `recall` for the *Churn* customers goes down, but same happens with the `precision` and, per se, with the `f1-score`. We can conclude that, when the sample increases, results go worse.


* **conclusions**

For the conclusions, as we discussed that, in some aspects, a reduced `test_size` improved certain parts of the model, it also happens that others drop. But here are the main aspects why **`test_size=0.1`** is better:
1. The percentage of `recall` is much better on that scenario.
2. Although `precision` is worse, what we are losing is less that what we are winning (overall `f1-score` improved).
3. `cohen_kappa_score` shows a tiny improvement with the reduced sample.

### Resampling

Although the measures take into consideration the `macro avg` instead of the `weighted avg`, it would be a good idea to see how the models performs with **resampling**. In this case, there will be two kind of resamples:

1. **Over Sampling**: Fake data will be created for the train set using the `SMOTE` method, increasing the size of the minority class as maximum as possible.
2. **Under Sampling**: Original data will be removed for the train set using `NearMiss` method, reducing the majority class to the same amount as the minority.

#### Over Sampling

In [88]:
# initializing SMOTE

smote = SMOTE()

X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# checking that the amount is the same for each value

print('Original dataset shape:', Counter(y_train))
print('Resample dataset shape:', Counter(y_train_smote))

Original dataset shape: Counter({1.0: 6799, 0.0: 1302})
Resample dataset shape: Counter({0.0: 6799, 1.0: 6799})


In [89]:
# Scaling the data
X_train_scaled_smote = scaler.fit_transform(X_train_smote)

In [90]:
# Obtaining best params
gradient_gridsearch(X_train_scaled_smote, y_train_smote)

GradientBoostingClassifier(criterion='mse', loss='exponential', max_depth=4,
                           max_features='sqrt', max_leaf_nodes=16,
                           n_estimators=265)

In [168]:
# Storing y_pred
y_pred_gradient_smote, class_report_gradient_smote, kappa_gradient_smote = gradient_trainer(X_train_scaled_smote,
                                                                                            X_test_scaled,
                                                                                            y_train_smote,
                                                                                            y_test,
                                                                                            "exponential",
                                                                                            "mse",
                                                                                            "sqrt",
                                                                                            265,
                                                                                            4,
                                                                                            16
                                                                                           )

#### Under Sampling

In [92]:
# initializing NearMiss

nm = NearMiss()

X_train_nm, y_train_nm = nm.fit_resample(X_train, y_train)

# checking that the amount is the same for each value

print('Original dataset shape:', Counter(y_train))
print('Resample dataset shape:', Counter(y_train_nm))

Original dataset shape: Counter({1.0: 6799, 0.0: 1302})
Resample dataset shape: Counter({0.0: 1302, 1.0: 1302})


In [93]:
# Scaling the data
X_train_scaled_nm = scaler.fit_transform(X_train_nm)

In [94]:
# Obtaining best params
gradient_gridsearch(X_train_scaled_nm, y_train_nm)

GradientBoostingClassifier(criterion='mse', max_depth=4, max_features='log2',
                           max_leaf_nodes=14, n_estimators=280)

In [169]:
# Storing y_pred
y_pred_gradient_nm, class_report_gradient_nm, kappa_gradient_nm = gradient_trainer(X_train_scaled_nm,
                                                                                   X_test_scaled,
                                                                                   y_train_nm,
                                                                                   y_test,
                                                                                   "deviance",
                                                                                   "mse",
                                                                                   "log2",
                                                                                   280,
                                                                                   4,
                                                                                   14
                                                                                  )

#### Results

In [170]:
# Classification Report
print(f"GradientBoost\ntest_size=0.1:\n{class_report_gradient_10}\n")
print(f"Over Sampling:\n{class_report_gradient_smote}\n")
print(f"Under Sampling:\n{class_report_gradient_nm}\n\n")

print(f"test_size=0.1 Kappa: {round(kappa_gradient_10,4)}")
print(f"Over Sampling Kappa: {round(kappa_gradient_smote,4)}")
print(f"Under Sampling Kappa: {round(kappa_gradient_nm,4)}")

GradientBoost
test_size=0.1:
                Churn    Customer  accuracy    macro avg  weighted avg
precision    0.920732    0.985866  0.975321     0.953299      0.975385
recall       0.926380    0.984706  0.975321     0.955543      0.975321
f1-score     0.923547    0.985285  0.975321     0.954416      0.975351
support    163.000000  850.000000  0.975321  1013.000000   1013.000000

Over Sampling:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.187104     1.000000   0.30306     0.593552      0.869600
recall       1.000000     0.169900   0.30306     0.584950      0.303060
f1-score     0.315228     0.290452   0.30306     0.302840      0.294427
support    325.000000  1701.000000   0.30306  2026.000000   2026.000000

Under Sampling:
                Churn     Customer  accuracy    macro avg  weighted avg
precision    0.269807     0.933150  0.627345     0.601479      0.826740
recall       0.775385     0.599059  0.627345     0.687222      0.627345
f1-scor

Looking the different results, we can conclude that neither **Over Sampling** or **Under Sampling** provides better results (or barely similar) to the previous ones. 

Although the `recall` for the *Churn* customers is perfect with the over sampling, the rest of variables are really bad, causing that the model has a very low `accuracy` and an even lower `cohen_kappa_score`.

On the other hand, the under sampling lowered the values mentioned on the previous paragraph, but not as aggresively as the over sampling does. Although, the model is still pretty bad and we will not consider it.

Last, but not least, we remain with the **GradientBoost | test_size = 0.1**